In [1]:
import requests
import csv
import json
import time
from datetime import datetime, timedelta
from collections import defaultdict

api_key = 'mzrsaV5E9ZwVPQ2biOQGyYansI7Ukh5p'


In [2]:
def print_json(json_data):
    print(json.dumps(json_data, indent=2))

def write_jsonl_file(data, filename):
    # Write data to a JSONL file
    with open(filename, 'a', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)
        file.write('\n')


In [3]:
def return_response(url) :
    response = requests.get(url)
    return response.json()

def generate_date_range(start_date, end_date):
    # Generate a date range between start_date and end_date
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += timedelta(weeks=1)


# Getting All Lists

In [7]:
output_filename = 'all_lists.json'
with open(output_filename, 'w', encoding='utf-8') as file:
    url = f'https://api.nytimes.com/svc/books/v3/lists/names.json?api-key={api_key}' # all lists
    response = return_response(url)

    json.dump(response, file, indent=4)
    print(f'All lists saved to {output_filename}.')


All lists saved to all_lists.json.


# Getting All Weeks

In [23]:
start_date_str = '2011-02-13'
end_date_str = '2023-12-10'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

all_responses = {}
failed_week = []

output_filename = 'all_weeks.jsonl'
with open(output_filename, 'a', encoding='utf-8') as file:
    for current_date in generate_date_range(start_date, end_date):
        time.sleep(12) #
        week = current_date.strftime('%Y-%m-%d')
        url = f'https://api.nytimes.com/svc/books/v3/lists/{week}/combined-print-and-e-book-fiction.json?api-key={api_key}'
        response = return_response(url)

        if ('status' in response and response['status']=='OK'):
            book = response['results']['books'][0]
            book = {
                'date': response['results']['published_date'],
                'rank': book['rank'],
                'title': book['title'],
                'author': book['author'],
                'primary_isbn10': book['primary_isbn10'],
                'dagger': book['dagger'],
            }
            month = book['date'].split("-")[0]+'-'+book['date'].split("-")[1]
            if month in all_responses:
                all_responses[month].append(book)
            else:
                all_responses[month] = [book]

            print(f'Response for week {week} added to the list.')

        else:
            print(week)
            failed_week.append(week)

    json.dump(all_responses, file, indent=2)


Response for week 2011-02-13 added to the list.
Response for week 2011-02-20 added to the list.
Response for week 2011-02-27 added to the list.
Response for week 2011-03-06 added to the list.
Response for week 2011-03-13 added to the list.
Response for week 2011-03-20 added to the list.
Response for week 2011-03-27 added to the list.
Response for week 2011-04-03 added to the list.
Response for week 2011-04-10 added to the list.
Response for week 2011-04-17 added to the list.
Response for week 2011-04-24 added to the list.
Response for week 2011-05-01 added to the list.
Response for week 2011-05-08 added to the list.
Response for week 2011-05-15 added to the list.
Response for week 2011-05-22 added to the list.
Response for week 2011-05-29 added to the list.
Response for week 2011-06-05 added to the list.
Response for week 2011-06-12 added to the list.
Response for week 2011-06-19 added to the list.
Response for week 2011-06-26 added to the list.
Response for week 2011-07-03 added to th

In [5]:
from dateutil.relativedelta import relativedelta
from collections import Counter

def generate_date_range_months(start_date, end_date):
    # Generate a date range between start_date and end_date
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += relativedelta(months=1)

start_date_str = '2011-02-13'
end_date_str = '2023-12-10'

start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

all_responses_month = {}

input_filename = 'all_books.json'
output_filename = 'all_months.jsonl'
with open(output_filename, 'w', encoding='utf-8') as file:
    with open(input_filename, 'r', encoding='utf-8') as read_file:
        data = json.load(input_filename)

        for current_date in generate_date_range_months(start_date, end_date):
            month = current_date.strftime('%Y-%m')
            book_list = data[month]

            titles = [book['title'] for book in book_list]
            title_counts = Counter(titles)
            most_common_title = title_counts.most_common(1)[0][0]

            most_common_book = next(book for book in book_list if book['title'] == most_common_title)
            all_responses_month[month] = most_common_book 
            
    json.dump(all_responses_month, file)


FileNotFoundError: [Errno 2] No such file or directory: 'all_books.jsonl'